In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [4]:
gunData = pd.read_excel('firearm_data_cleaned_new.xlsx')
gunData.head()

,year,state,rate,deaths,state_name,law_strength_score,restrictive_laws,permissive_laws,total_law_changes,rate_change,...,strength_firearm_removal_at_scene_of_domestic_violence,strength_firearms_in_college_university,strength_child_access_laws,strength_gun_trafficking,strength_open_carry,strength_required_reporting_of_lost_or_stolen_firearms,strength_safety_training_required,strength_untraceable_firearms,strength_permit_to_purchase,strength_firearms_in_k_12_educational_settings
0,2014,AK,19.2,145,Alaska,11,18,7,25,NaN,...,1,0,0,0,0,0,0,0,0,0
1,2015,AK,23.4,177,Alaska,11,18,7,25,4.2,...,1,0,0,0,0,0,0,0,0,0
2,2016,AK,23.3,177,Alaska,11,18,7,25,-0.1,...,1,0,0,0,0,0,0,0,0,0
3,2017,AK,24.5,180,Alaska,11,18,7,25,1.2,...,1,0,0,0,0,0,0,0,0,0
4,2018,AK,21.0,155,Alaska,11,18,7,25,-3.5,...,1,0,0,0,0,0,0,0,0,0


In [5]:
gunData.shape

(502, 32)

In [4]:
""""
502 rows by 32 columns:

'year', 
'state', 
'rate', 
'deaths', 
'state_name', 
'law_strength_score',
'restrictive_laws', 
'permissive_laws', 
'total_law_changes',
'rate_change',                              # 10
'law_strength_change', 
'unique_law_classes',
'strength_background_checks',
'strength_carrying_a_concealed_weapon_ccw', 
'strength_castle_doctrine',
'strength_dealer_license', 
'strength_firearm_sales_restrictions',
'strength_local_laws_preempted_by_state', 
'strength_minimum_age',
'strength_prohibited_possessor',            # 20
'strength_registration',
'strength_waiting_period',
'strength_firearm_removal_at_scene_of_domestic_violence',
'strength_firearms_in_college_university', 
'strength_child_access_laws',
'strength_gun_trafficking', 
'strength_open_carry',
'strength_required_reporting_of_lost_or_stolen_firearms',
'strength_safety_training_required', 
'strength_untraceable_firearms',            # 30
'strength_permit_to_purchase',
'strength_firearms_in_k_12_educational_settings'
"""

'"\n502 rows by 32 columns:\n\n\'year\', \n\'state\', \n\'rate\', \n\'deaths\', \n\'state_name\', \n\'law_strength_score\',\n\'restrictive_laws\', \n\'permissive_laws\', \n\'total_law_changes\',\n\'rate_change\',                              # 10\n\'law_strength_change\', \n\'unique_law_classes\',\n\'strength_background_checks\',\n\'strength_carrying_a_concealed_weapon_ccw\', \n\'strength_castle_doctrine\',\n\'strength_dealer_license\', \n\'strength_firearm_sales_restrictions\',\n\'strength_local_laws_preempted_by_state\', \n\'strength_minimum_age\',\n\'strength_prohibited_possessor\',            # 20\n\'strength_registration\',\n\'strength_waiting_period\',\n\'strength_firearm_removal_at_scene_of_domestic_violence\',\n\'strength_firearms_in_college_university\', \n\'strength_child_access_laws\',\n\'strength_gun_trafficking\', \n\'strength_open_carry\',\n\'strength_required_reporting_of_lost_or_stolen_firearms\',\n\'strength_safety_training_required\', \n\'strength_untraceable_firearms

## RESEARCH QUESTION 1:

_Can we predict firearm death rates based on gun law characteristics?_

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge

In [6]:
# kitchen sink models

data1 = gunData[['year', 'state', 'rate', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']].dropna()
    # state is enough, omitting state_name
    # omitting rate_change
data1 = data1.query("state!='District of Columbia'")
    # there is only one datapoint for District of Columbia, flummoxes train_test_split. dropping.


data2 = gunData[['rate', 'state', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']].dropna()
data2 = data2.query("state!='District of Columbia'")

## 4x models:
* Multiple Regression (sensible)
* Logit (Does not make sense, we are not looking for a binary outcome)
* KNN >> evaluate to see if rate breaks into distinct clusters? >> can we do multiple predictors in this?
* K Means >> evaluate to see if rate breaks into distinct clusters? (on its own)

In [7]:
bigX_1 = data1[['year', 'state', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']]
    # state is enough, omitting state_name
    # omitting rate_change

bigX_2 = data2[['state', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']]
bigY1 = data1['rate']
bigY2 = data2['rate']

categories1 = ['year', 'state']
numers1 = ['deaths', 'law_strength_score', 'restrictive_laws', 'permissive_laws', 'total_law_changes',                           
                    'law_strength_change', 'unique_law_classes']

categories2 = ['state']
numers2 = ['strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']

In [8]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(bigX_1, bigY1, test_size = 0.2, random_state = 123)
x_train2, x_test2, y_train2, y_test2 = train_test_split(bigX_2, bigY2, test_size = 0.2, random_state = 123)

In [9]:
xform1 = ColumnTransformer(transformers = [("encoder1", OneHotEncoder(drop='first'), categories1),
                                           ("numeric1", "passthrough", numers1)])
xform2 = ColumnTransformer(transformers = [("encoder2", OneHotEncoder(drop='first'), categories2),
                                           ("numeric2", "passthrough", numers2)])

In [10]:
sinkMod1 = Pipeline(steps = [("transformer1", xform1), ("model1", LinearRegression())])
sinkMod2 = Pipeline(steps = [("transformer2", xform2), ("model2", LinearRegression())])

In [11]:
sinkMod1.fit(x_train1, y_train1)
sinkMod2.fit(x_train2, y_train2)

,steps,"[('transformer2', ...), ('model2', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('encoder2', ...), ('numeric2', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [12]:
predictions1 = sinkMod1.predict(x_test1)
mse_calc1 = mean_squared_error(y_test1, predictions1)
root_mse1 = mse_calc1 ** 0.5
root_mse1

1.2767631486908197

In [13]:
predictions2 = sinkMod2.predict(x_test2)
mse_calc2 = mean_squared_error(y_test2, predictions2)
root_mse2 = mse_calc2 ** 0.5
root_mse2



1.5975319060496322

In [14]:
"""
the following code (calculating adjusted r^2) was produced by generative AI (Claude by Anthropic)) 
in response to a direct request of how to calculate adjusted r^2 for a multi-linear model built with
scikitlearn's Pipeline.
"""

r2_1 = sinkMod1.score(x_test1, y_test1)
n_1 = x_test1.shape[0]
p_1 = x_test1.shape[1]
adj_r2_1 = 1 - (1 - r2_1) * (n_1-1) / (n_1 - p_1 - 1)
adj_r2_1

0.9462662882466966

In [15]:
r2_2 = sinkMod2.score(x_test2, y_test2)
n_2 = x_test2.shape[0]
p_2 = x_test2.shape[1]
adj_r2_2 = 1 - (1 - r2_2) * (n_2 - 1) / (n_2 - p_2 -1)
adj_r2_2

0.903951369034542

In [16]:
"""
the following code (capturing coefficients from the model and putting them into a dataframe, the latter several cells down) was 
produced by generative AI (Claude by Anthropic)) in response to a direct request of how to determine coefficients for 
a multi-linear model built with scikitlearn's OneHotEncoder.
"""

first_set = xform1.get_feature_names_out()
second_set = xform2.get_feature_names_out()

In [17]:
firstMod = sinkMod1.named_steps['model1']
print(f"Intercept for first model: {firstMod.intercept_}")
print(f"Coefficients for first model: {firstMod.coef_}")

Intercept for first model: 20.125028214006463
Coefficients for first model: [ 4.40071645e-01  6.76542356e-01  6.23779317e-01  7.75806104e-01
  2.08719224e+00  2.76382044e+00  2.24115948e+00  2.01179436e+00
 -4.52235686e+00 -5.14583740e+00 -1.10307586e+01 -2.27215290e+01
 -1.05875030e+01 -1.47541625e+01 -8.41558813e+00 -2.29150563e+01
 -1.46492540e+01 -1.25140953e+01 -1.50409268e+01 -7.23409010e+00
 -1.23399581e+01 -1.18279320e+01 -1.02805411e+01 -9.10216186e+00
 -3.78197018e+00 -1.63197075e+01 -1.18971313e+01 -1.18291999e+01
 -1.51745512e+01 -1.68303462e+01 -8.36272396e+00 -7.92478036e-01
 -4.94927342e+00 -1.45539073e+01 -1.03112657e+01 -1.33715111e+01
 -1.47680637e+01 -1.63157765e+01 -2.53329235e+00 -6.34648018e+00
 -1.89264140e+01 -1.59287694e+01 -7.89189009e+00 -1.09246463e+01
 -1.73121089e+01 -1.77174126e+01 -7.82995739e+00 -1.10977393e+01
 -1.09442349e+01 -2.97197955e+01 -1.08910247e+01 -1.46981204e+01
 -1.14716587e+01 -1.38281549e+01 -1.38398799e+01 -6.80444312e+00
 -2.66841157e+

In [18]:
secondMod = sinkMod2.named_steps['model2']
print(f"Intercept for second model: {secondMod.intercept_}")
print(f"Coefficients for second model: {secondMod.coef_}")

Intercept for second model: 16.726559713111094
Coefficients for second model: [-12.08978922   3.10137768  -6.86703928 -33.03103738  -8.81701635
 -34.26271782 -10.3505908   -4.02325555  -9.93355934 -44.56180847
 -13.83043223  -6.41319458 -10.88207494  -5.24932697  -6.85678186
  -5.13473502   4.22563924 -19.80444036 -22.15100367 -12.37900063
 -19.61357131 -12.86968469 -12.46408638   4.01795081  -4.82734399
 -14.87594211  -8.96209385  -5.22836789 -21.19913791 -21.87051295
  12.75234354  -5.05188144 -21.4198798   -3.54672044   2.93657481
  -7.72915408 -13.9913036  -20.91940613   0.68883371 -12.73795239
  -0.92763936  -6.09884712 -12.70675995 -11.28569973  -7.67112832
 -23.60477446  -9.55829128  -6.25990498  -1.39241677   0.27485443
  -1.38710613  -1.60253671   2.8321995   -0.51469793  -1.61296271
  -0.84194301   0.14681205   5.71626037   0.5554692    2.15914812
  -1.90463346   0.84089533   0.53496469   0.44791574   0.16525245
  -3.34514214   0.23885331   0.92684537  -2.24859407]


In [19]:
coef_df_1 = pd.DataFrame({'feature': first_set, 'coefficient': firstMod.coef_})
coef_df_1

,feature,coefficient
0,encoder1__year_2016,0.440072
1,encoder1__year_2017,0.676542
2,encoder1__year_2018,0.623779
3,encoder1__year_2019,0.775806
4,encoder1__year_2020,2.087192
...,...,...
59,numeric1__restrictive_laws,-0.070956
60,numeric1__permissive_laws,0.040369
61,numeric1__total_law_changes,-0.030587
62,numeric1__law_strength_change,0.086217


In [20]:
coef_df_2 = pd.DataFrame({'feature': second_set, 'coefficient': secondMod.coef_})
coef_df_2

,feature,coefficient
0,encoder2__state_AL,-12.089789
1,encoder2__state_AR,3.101378
2,encoder2__state_AZ,-6.867039
3,encoder2__state_CA,-33.031037
4,encoder2__state_CO,-8.817016
...,...,...
64,numeric2__strength_required_reporting_of_lost_...,0.165252
65,numeric2__strength_safety_training_required,-3.345142
66,numeric2__strength_untraceable_firearms,0.238853
67,numeric2__strength_permit_to_purchase,0.926845


In [21]:
plotData1 = pd.DataFrame({"actual": y_test1, "predictions": predictions1})

plotData2 = pd.DataFrame({"actual": y_test2, "predictions": predictions2})

In [22]:
px.scatter(plotData1, x="actual", y="predictions", trendline='ols')


In [23]:
px.scatter(plotData2, x="actual", y="predictions", trendline='ols')

In [ ]:
"""
Apply Ridge regression (IAW notebook 12)

1.  Plot the lambdas like the notebook indicates
2.  Calculate the RMSE for each lambda value
3.  Select the best lambda value >> harvest the coefficients and calculate R^2

NOTE: doing Ridge but NOT Lasso because Lasso is strong when only a few predictors matter;
This project attempts to model a complex phenomenon, so this possiblity is dismissed.

"""

# Ridge regression with multiple alpha (lambda) values
alphas = np.logspace(-2, 6, 100)
ridge_coeffs = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(bigX_1, bigY1) # <<<<<<<<<<<<<<< this is not right
    ridge_coeffs.append(ridge.coef_)

ridge_coeffs = np.array(ridge_coeffs)

# Plot coefficient paths
plt.figure(figsize=(12, 6))
for i in range(X.shape[1]):
    plt.plot(np.log(alphas), ridge_coeffs[:, i], label=diabetes.feature_names[i])
plt.xlabel('log(lambda)')
plt.ylabel('Coefficients')
plt.title('Ridge Regression Coefficient Paths')
plt.legend(loc='best', fontsize=8)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Get coefficients at specific lambda
lnlam = 2
ridge_specific = Ridge(alpha=np.exp(lnlam))
ridge_specific.fit(X, y)
print(f"\nRidge coefficients at log(lambda)={lnlam}:")
print(pd.DataFrame({'feature': diabetes.feature_names,
                   'ridge': ridge_specific.coef_,
                   'ols': mod_me.coef_}))